In [2]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

C:\Python36\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from sklearn import model_selection

In [4]:
from sklearn import cross_validation

In [5]:
test = pd.read_csv("./input/test.csv")   # test set
traindf = pd.read_csv("./input/train.csv")
#sampledf = pd.read_csv("./input/gender_submission.csv")

PassengerId = test['PassengerId']

In [214]:
test.shape

(418, 16)

In [215]:
traindf.shape

(891, 17)

In [6]:
full_data = [traindf, test]

In [7]:
traindf['Age'] = traindf['Age'].replace(np.nan, traindf.Age.median())
test['Age'] = test['Age'].replace(np.nan, traindf.Age.median())

for dataset in full_data:
    dataset['familySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['familySize'] == 1, 'IsAlone'] = 1
    
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
    
for dataset in full_data:
    dataset['Fare'] = dataset['Fare'].fillna(traindf['Fare'].median())
    
def k_cabin(x):
    x = str(x)
    cabin_search = re.search('([A-L]+)', x)
    if cabin_search:
        return cabin_search.group(1)
    return "None"

for dataset in full_data:
    dataset['k_cabin'] = dataset['Cabin'].apply(k_cabin)

In [8]:
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""
# Create a new feature Title, containing the titles of passenger names
for dataset in full_data:
    dataset['Title'] = dataset['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"
for dataset in full_data:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

In [9]:
for dataset in full_data:
    # Mapping titles
    title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [10]:
traindf.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,familySize,IsAlone,k_cabin,Title
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,2,0,None,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,2,0,C,3
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,1,None,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,2,0,C,3
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1,1,None,1


In [11]:
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    

In [12]:
# 더미화
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

traindf['c_cabin'] = encoder.fit_transform(traindf['k_cabin'])
test['c_cabin'] = encoder.fit_transform(test['k_cabin'])

In [13]:
cols = ['Pclass', 'Sex', 'Age', 'Fare', 'familySize', 'Embarked', 'IsAlone', 'c_cabin', 'Title']
target = ['Survived']

In [14]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [15]:
lr.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [16]:
lr.score(traindf[cols], traindf[target])

0.81481481481481477

In [188]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5)

In [189]:
clf = clf.fit(traindf[cols], traindf[target])

In [190]:
clf.score(traindf[cols], traindf[target])

0.84848484848484851

In [191]:
pd.Series(clf.feature_importances_, index=cols)

Sex           0.561990
Age           0.096737
Fare          0.131486
familySize    0.122916
c_cabin       0.086871
dtype: float64

In [192]:
from sklearn.ensemble import BaggingClassifier

In [193]:
bagging = BaggingClassifier(DecisionTreeClassifier(), 
                            max_samples=0.5, max_features=0.5)

In [194]:
bagging.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.5, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [195]:
bagging.score(traindf[cols], traindf[target])

0.84736251402918072

In [196]:
from sklearn.neighbors import KNeighborsClassifier

In [197]:
baggingKN = BaggingClassifier(KNeighborsClassifier(), 
                            max_samples=0.5, max_features=0.5)

In [198]:
baggingKN.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.5, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [199]:
baggingKN.score(traindf[cols], traindf[target])

0.83052749719416386

In [200]:
from sklearn import svm
svmex = svm.SVC()
svmex.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [201]:
svmex.score(traindf[cols], traindf[target])

0.9135802469135802

In [31]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=30, max_depth=24)

In [32]:
rfc.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=24, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=30, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [33]:
rfc.score(traindf[cols], traindf[target])

0.978675645342312

In [34]:
# feature importance
rfc.feature_importances_

array([ 0.07678712,  0.14070276,  0.20511912,  0.22487484,  0.06092692,
        0.02557996,  0.0108425 ,  0.05677988,  0.19838691])

In [35]:
pd.Series(rfc.feature_importances_, index=cols)

Pclass        0.076787
Sex           0.140703
Age           0.205119
Fare          0.224875
familySize    0.060927
Embarked      0.025580
IsAlone       0.010842
c_cabin       0.056780
Title         0.198387
dtype: float64

In [36]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
adbst = AdaBoostRegressor(DecisionTreeRegressor(max_depth=5),
                          n_estimators=300, random_state=1)

In [37]:
adbst.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostRegressor(base_estimator=DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_split=1e-07,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best'),
         learning_rate=1.0, loss='linear', n_estimators=300,
         random_state=1)

In [38]:
adbst.score(traindf[cols], traindf[target])

0.49449057825514275

In [39]:
xgbr = xgb.XGBClassifier(
    learning_rate = 0.3, 
    n_estimators= 7000, max_depth=4, min_child_weight= 1, 
    gamma=0,
    #subsample=0.8, 
    #colsample_bytree=0.8, 
    objective= 'binary:logistic',
    scale_pos_weight=1)

In [40]:
xgbr.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\preprocessing\label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Python36\lib\site-packages\sklearn\preprocessing\label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.3, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=7000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [41]:
xgbr.score(traindf[cols], traindf[target])

0.98316498316498313

## Doing F1 Scoring

In [42]:
#xg_train_prediction = Prediction(xgbr, train_x)
dct_prd = clf.predict(traindf[cols])
bagging_prd = bagging.predict(traindf[cols])
baggingKN_prd = baggingKN.predict(traindf[cols])
svm_prd = svmex.predict(traindf[cols])
rfc_prd = rfc.predict(traindf[cols])
adbst_prd = adbst.predict(traindf[cols])
xgbr_prd = xgbr.predict(traindf[cols])
lr_prd = lr.predict(traindf[cols])

In [43]:
from sklearn.metrics import f1_score
f1_score(traindf[target], dct_prd, average = 'binary')

0.80118694362017806

In [44]:
f1_score(traindf[target], bagging_prd, average = 'binary')

0.84792626728110598

In [45]:
f1_score(traindf[target], baggingKN_prd, average = 'binary')

0.79815100154083196

In [46]:
f1_score(traindf[target], svm_prd, average = 'binary')

0.87841945288753798

In [47]:
f1_score(traindf[target], rfc_prd, average = 'binary')

0.97193500738552441

In [48]:
f1_score(traindf[target], xgbr_prd, average = 'binary')

0.97797356828193827

In [49]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=30, max_depth=24)

rfc.fit(traindf[cols], traindf[target])indf[target], lr_prd, average = 'binary')

0.75262368815592207

In [50]:
traindf[cols].shape

(891, 9)

In [51]:
traindf[target].shape

(891, 1)

In [52]:
# http://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

In [176]:
cols = [#'Pclass', 
        'Sex', 
        'Age', 
        'Fare', 
        #'Embarked', 
        'familySize', 
        #'IsAlone',
        'c_cabin', 
        #'Title'
       ]

In [177]:
xgbr = xgb.XGBClassifier(
    learning_rate = 0.1, 
    n_estimators= 200, max_depth=3,
    gamma=0,
    subsample=0.8, 
    #colsample_bytree=0.8, 
    objective= 'binary:logistic', 
    reg_lambda=0.3,
    scale_pos_weight=1)

xgbr.fit(traindf[cols], traindf[target])

xgbr.score(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\sklearn\preprocessing\label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Python36\lib\site-packages\sklearn\preprocessing\label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.90011223344556679

In [178]:
from sklearn.model_selection import cross_val_score
model = [clf, bagging, baggingKN, svmex, rfc, xgbr, lr]
scores = cross_val_score(model[5], traindf[cols], traindf['Survived'], 
                         cv=5, scoring='accuracy')
scores.mean()

0.83504282415657638

In [217]:
cols = ['Pclass', 
        'Sex', 
        'Age', 
        'Fare', 
        #'Embarked', 
        'familySize', 
        'IsAlone',
        #'c_cabin', 
        #'Title'
       ]

In [218]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=35, max_depth=10)

In [219]:
rfc.fit(traindf[cols], traindf[target])

C:\Python36\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=35, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [220]:
from sklearn.model_selection import cross_val_score
model = [clf, bagging, baggingKN, svmex, rfc, xgbr, lr]
scores = cross_val_score(model[4], traindf[cols], traindf['Survived'], 
                         cv=3, scoring='accuracy')
scores.mean()

0.81481481481481488

In [56]:
for i in model:
    scores = cross_val_score(i, traindf[cols], traindf['Survived'], 
                         cv=5, scoring='accuracy')
    print(scores.mean())
    print

0.819356568422
0.810386493732
0.788033706092
0.719556441674
0.81483051734
0.849656126509
0.795678609767


In [202]:
predictions = clf.predict(test[cols])

In [203]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [204]:
StackingSubmission.to_csv("Submission_dct.csv", index=False)

In [205]:
predictions = bagging.predict(test[cols])

In [206]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [207]:
StackingSubmission.to_csv("Submission_bag.csv", index=False)

In [208]:
predictions = baggingKN.predict(test[cols])

In [209]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [210]:
StackingSubmission.to_csv("Submission_bagKN.csv", index=False)

In [211]:
predictions = svmex.predict(test[cols])

In [212]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [213]:
StackingSubmission.to_csv("Submission_svm.csv", index=False)

In [182]:
predictions = rfc.predict(test[cols])

In [183]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [184]:
StackingSubmission.to_csv("Submission_rfc.csv", index=False)

In [185]:
predictions = xgbr.predict(test[cols])

In [186]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [187]:
StackingSubmission.to_csv("Submission_xgb.csv", index=False)

In [173]:
predictions = lr.predict(test[cols])

In [174]:
StackingSubmission = pd.DataFrame({ 'PassengerId': PassengerId,
                            'Survived': predictions })

In [175]:
StackingSubmission.to_csv("Submission_lr.csv", index=False)

In [126]:
col = ['Sex', 'Age', 'Fare', 'Embarked', 'familySize']

In [47]:
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import learn
from sklearn.metrics import accuracy_score

import random


In [190]:
classifier = learn.DNNClassifier(hidden_units=[20, 40, 20, 10],
    n_classes=2,
    feature_columns=learn.infer_real_valued_columns_from_input(traindf[col]),
    optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.06))

classifier.fit(traindf[col], traindf[target], batch_size=32, steps=150)

print(accuracy_score(list(classifier.predict(traindf[col])), traindf[target]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022049560550>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Sanghun\\AppData\\Local\\Temp\\tmprfzzq1g8'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instruct

In [197]:
# 3 layer neural network with hyperbolic tangent activation.

def dnn_tanh(features, target):
    target = tf.one_hot(target, 2, 1.0, 0.0)
    logits = layers.stack(features, layers.fully_connected, [10, 20, 40, 10],
        activation_fn=tf.tanh)
    prediction, loss = learn.models.logistic_regression(logits, target)
    train_op = layers.optimize_loss(loss,
        tf.contrib.framework.get_global_step(), optimizer='SGD', learning_rate=0.06)
    return tf.argmax(prediction, dimension=1), loss, train_op

random.seed(42)
classifier = learn.Estimator(model_fn=dnn_tanh)
classifier.fit(traindf[col], traindf[target], batch_size=16, steps=150)
print(accuracy_score(list(classifier.predict(traindf[col])), traindf[target]))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000022048FB82B0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\Sanghun\\AppData\\Local\\Temp\\tmpcmhzq4i9'}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instruct